<a href="https://colab.research.google.com/github/Milk448/.github.io/blob/master/Project_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 86.1 MB/s eta 0:00:00


In [2]:
# --- API Key Rotator (robust, cooldown-aware) ---
import os
import time
from threading import Lock
from dotenv import load_dotenv
import google.generativeai as genai
from PIL import Image
import json
import csv
import re
import logging
import time
import math
from datetime import datetime
from pathlib import Path
from PIL import Image, ImageEnhance, ImageFilter
import google.generativeai as genai
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import fitz  # PyMuPDF
load_dotenv()

False

In [3]:
import os

# Create the input directory if it doesn't exist
input_dir = '/content/input'
os.makedirs(input_dir, exist_ok=True)

# Upload files
from google.colab import files
uploaded = files.upload()

# Move uploaded files to /content/input
for filename in uploaded.keys():
    os.rename(filename, os.path.join(input_dir, filename))

Saving hard and compelx Q&As_20.pdf to hard and compelx Q&As_20.pdf


In [ ]:
import os
import time
from threading import Lock
from concurrent.futures import ThreadPoolExecutor, as_completed
import google.generativeai as genai
from PIL import Image, ImageEnhance, ImageFilter
import json
import csv
import re
import logging
import math
from datetime import datetime
from pathlib import Path
import fitz  # PyMuPDF
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import userdata

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - INFO: %(message)s',
    handlers=[logging.StreamHandler()]
)

# Configuration
SCRIPT_DIR = os.getcwd()
INPUT_DIR = os.path.join(SCRIPT_DIR, "input")
OUTPUT_DIR = os.path.join(SCRIPT_DIR, "output")
RAW_OUTPUT = os.path.join(OUTPUT_DIR, "raw_output")
TEMP_IMG_DIR = os.path.join(SCRIPT_DIR, "temp_img")
OUTPUT_CSV = os.path.join(OUTPUT_DIR, "output_results.csv")
OUTPUT_JSON = os.path.join(OUTPUT_DIR, "output_results.json")
OUTPUT_PIPELINE_METRICS = os.path.join(OUTPUT_DIR, "pipeline_metrics.png")
MAX_PROBLEMS = 100
PDF_PAGE_SELECTION = "all"

# Create directories if they don't exist
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(RAW_OUTPUT, exist_ok=True)
os.makedirs(TEMP_IMG_DIR, exist_ok=True)

# --- API Key Rotator ---
class APIKeyRotator:
    def __init__(self, env_var='GEMINI_API_KEYS', max_requests_per_minute=5, cooldown_seconds=300):
        try:
            keys = userdata.get(env_var).split(',')
        except:
            keys = os.getenv(env_var, '').split(',')
        self.api_keys = [k.strip() for k in keys if k.strip()]
        if not self.api_keys:
            logging.error("No valid API keys found in GEMINI_API_KEYS. Please set it in Colab Secrets or environment.")
            exit(1)
        self.max_requests = max_requests_per_minute
        self.cooldown_seconds = cooldown_seconds
        self.usage = [0] * len(self.api_keys)
        self.timestamps = [time.time()] * len(self.api_keys)
        self.cooldown = [0] * len(self.api_keys)
        self.lock = Lock()
        self.index = 0
        logging.info(f"Initialized APIKeyRotator with {len(self.api_keys)} keys: {[k[:5] + '...' for k in self.api_keys]}")
        self.validate_keys()

    def validate_keys(self):
        """Validate each API key by making a test request."""
        for idx, key in enumerate(self.api_keys):
            try:
                genai.configure(api_key=key)
                model = genai.GenerativeModel('gemini-2.0-flash')
                response = model.generate_content("Test prompt")
                logging.info(f"API key {idx} is valid.")
            except Exception as e:
                logging.warning(f"API key {idx} may be invalid or rate-limited: {e}")
                self.mark_bad_key(idx, retry_after=300)

    def get_key(self):
        with self.lock:
            now = time.time()
            logging.info(f"Checking {len(self.api_keys)} API keys. Usage: {self.usage}, Cooldowns: {[round(c - now, 2) for c in self.cooldown]}")
            for _ in range(len(self.api_keys)):
                idx = self.index
                if now - self.timestamps[idx] > 60:
                    logging.info(f"Resetting usage for key {idx}: {self.usage[idx]} -> 0")
                    self.usage[idx] = 0
                    self.timestamps[idx] = now
                if now < self.cooldown[idx]:
                    logging.info(f"Key {idx} in cooldown until {self.cooldown[idx]}")
                    self.index = (idx + 1) % len(self.api_keys)
                    continue
                if self.usage[idx] < self.max_requests:
                    self.usage[idx] += 1
                    logging.info(f"Using key {idx} (usage: {self.usage[idx]}/{self.max_requests})")
                    self.index = (idx + 1) % len(self.api_keys)
                    return self.api_keys[idx], idx
                self.index = (idx + 1) % len(self.api_keys)
            min_cooldown = min([c - now for c in self.cooldown if c > now], default=self.cooldown_seconds)
            logging.info(f"All keys in cooldown. Waiting {min_cooldown:.2f} seconds...")
            time.sleep(min_cooldown)
            raise Exception("All API keys are in cooldown or over quota. Retrying after wait.")

    def mark_bad_key(self, idx, retry_after=None):
        with self.lock:
            cooldown_time = retry_after if retry_after else self.cooldown_seconds
            self.cooldown[idx] = time.time() + cooldown_time
            logging.info(f"Marked key {idx} as bad. Cooldown until {self.cooldown[idx]} (duration: {cooldown_time}s)")

api_key_rotator = APIKeyRotator(max_requests_per_minute=5, cooldown_seconds=300)

# --- Utility Functions ---
def parse_page_selection(page_selection, max_pages):
    """Parse page selection string into list of page numbers (0-based)."""
    if page_selection.lower() == "all":
        return list(range(max_pages))
    pages = set()
    parts = page_selection.split(',')
    for part in parts:
        part = part.strip()
        if not part:
            continue
        if '-' in part:
            try:
                start, end = map(int, part.split('-'))
                pages.update(range(start - 1, end))
            except ValueError:
                logging.warning(f"Invalid page range format: {part}. Skipping.")
        else:
            try:
                pages.add(int(part) - 1)
            except ValueError:
                logging.warning(f"Invalid page number format: {part}. Skipping.")
    return sorted([p for p in pages if 0 <= p < max_pages])

def clean_json_response(response_text: str) -> str:
    """Robust JSON cleaner for Gemini-2.0-flash output with complex LaTeX."""
    json_str = response_text.strip()
    extraction_patterns = [
        r'(?s)```json\n(.*?)\n```',
        r'(?s)\{(.*)\}',
        r'(?s)"generated_question_llm1":.*?\}',
        r'(?s)\{(.*?)(?<!\\)\}',
    ]
    for pattern in extraction_patterns:
        try:
            match = re.search(pattern, json_str)
            if match:
                json_str = match.group(1 if pattern == extraction_patterns[0] else 0).strip()
                break
        except:
            continue
    protected_patterns = [
        (r'\\\\', r'ⓢⓢ'), (r'\\"', r'ⓠ'), (r'\\/', r'ⓕ'), (r'\\b', r'ⓑ'),
        (r'\\n', r'ⓝ'), (r'\\r', r'ⓡ'), (r'\\t', r'ⓣ')
    ]
    for pattern, replacement in protected_patterns:
        json_str = re.sub(pattern, replacement, json_str)
    json_str = re.sub(r'\\(?![ⓢⓠⓕⓑⓝⓡⓣ])', r'\\\\', json_str)
    restoration_patterns = [
        (r'ⓢⓢ', r'\\\\'), (r'ⓠ', r'\\"'), (r'ⓕ', r'\\/'), (r'ⓑ', r'\\b'),
        (r'ⓝ', r'\\n'), (r'ⓡ', r'\\r'), (r'ⓣ', r'\\t')
    ]
    for pattern, replacement in restoration_patterns:
        json_str = json_str.replace(pattern, replacement)
    math_modes = [
        (r'\\\(', r'ⓛⓟ'), (r'\\\)', r'ⓛⓡ'), (r'\\\[', r'ⓓⓛ'), (r'\\\]', r'ⓓⓡ'),
        (r'\$\$(.*?)\$\$', r'ⓓⓓ\1ⓓⓓ'), (r'\$(.*?)\$', r'ⓘ\1ⓘ')
    ]
    for pattern, replacement in math_modes:
        json_str = re.sub(pattern, replacement, json_str)
    json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
    json_str = ''.join(char for char in json_str if ord(char) >= 32 or char in '\t\n\r')
    math_restore = [
        (r'ⓛⓟ', r'\\('), (r'ⓛⓡ', r'\\)'), (r'ⓓⓛ', r'\\['), (r'ⓓⓡ', r'\\]'),
        (r'ⓓⓓ(.*?)ⓓⓓ', r'$$\1$$'), (r'ⓘ(.*?)ⓘ', r'$\1$')
    ]
    for pattern, replacement in math_restore:
        json_str = json_str.replace(pattern, replacement)
    if not json_str.startswith('{'):
        json_str = '{' + json_str
    if not json_str.endswith('}'):
        json_str += '}'
    return json_str

class ProblemVisualizer:
    def __init__(self, output_dir):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)

    def generate_metrics_chart(self, results):
        """Generates four subplots: processing status, evaluation accuracy, taxonomy success, and token usage."""
        valid_results = [r for r in results if isinstance(r, dict) and 'status' not in r]
        if not valid_results:
            logging.info("[!] No valid results to visualize.")
            return
        image_filenames_for_labels = [r.get('image_filename', f"Problem {i+1}") for i, r in enumerate(results)]
        success = [1 if isinstance(r, dict) and 'status' not in r else 0 for r in results]
        failed = [1 if isinstance(r, dict) and 'status' in r else 0 for r in results]
        correct = [1 if r.get('judge_evaluation') == "Correct" else 0 for r in valid_results]
        incorrect = [1 if r.get('judge_evaluation') == "Incorrect" else 0 for r in valid_results]
        total_evaluated = len(correct)
        accuracy = (sum(correct) / total_evaluated * 100) if total_evaluated > 0 else 0
        valid_subcategory = [1 if r.get('subcategory', 'N/A') != "N/A" else 0 for r in valid_results]
        invalid_subcategory = [1 if r.get('subcategory', 'N/A') == "N/A" else 0 for r in valid_results]
        prompt_tokens = [r.get('total_prompt_tokens_per_question', 0) for r in valid_results]
        candidate_tokens = [r.get('total_candidate_tokens_per_question', 0) for r in valid_results]
        successful_qnas = sum(success)
        total_attempts = len(results)
        success_rate = (successful_qnas / total_attempts * 100) if total_attempts > 0 else 0

        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        x = np.arange(len(results))
        width = 0.35
        ax1.bar(x - width/2, success, width, label=f'Success ({success_rate:.1f}%)', color='#4CAF50')
        ax1.bar(x + width/2, failed, width, label=f'Failed ({100 - success_rate:.1f}%)', color='#FF6B6B')
        ax1.set_ylabel('Count')
        ax1.set_title(f'Processing Status (Successful Q&As: {successful_qnas}/{total_attempts})')
        ax1.legend()
        ax1.set_xticks(x)
        ax1.set_xticklabels(image_filenames_for_labels, rotation=45, ha='right')
        ax1.set_ylim(0, 1.2)

        x_valid = np.arange(len(valid_results))
        ax2.bar(x_valid - width/2, correct, width, label='Correct', color='#4CAF50')
        ax2.bar(x_valid + width/2, incorrect, width, label='Incorrect', color='#FF6B6B')
        ax2.set_ylabel('Count')
        ax2.set_title(f'Evaluation Accuracy ({accuracy:.1f}% Correct, {total_evaluated} Evaluated)')
        ax2.legend()
        ax2.set_xticks(x_valid)
        ax2.set_xticklabels([r['image_filename'] for r in valid_results], rotation=45, ha='right')
        ax2.set_ylim(0, max(max(correct + [0]), max(incorrect + [0])) + 0.2)

        ax3.bar(x_valid - width/2, valid_subcategory, width, label='Valid Subcategory', color='#4CAF50')
        ax3.bar(x_valid + width/2, invalid_subcategory, width, label='Invalid Subcategory (N/A)', color='#FF6B6B')
        ax3.set_ylabel('Count')
        ax3.set_title(f'Taxonomy Classification (Valid: {sum(valid_subcategory)}/{len(valid_results)})')
        ax3.legend()
        ax3.set_xticks(x_valid)
        ax3.set_xticklabels([r['image_filename'] for r in valid_results], rotation=45, ha='right')
        ax3.set_ylim(0, max(max(valid_subcategory + [0]), max(invalid_subcategory + [0])) + 0.2)

        ax4.bar(x_valid - width/2, prompt_tokens, width, label='Prompt Tokens', color='#4CAF50')
        ax4.bar(x_valid + width/2, candidate_tokens, width, label='Candidate Tokens', color='#0288D1')
        ax4.set_ylabel('Tokens')
        ax4.set_title('Token Usage per Problem')
        ax4.legend()
        ax4.set_xticks(x_valid)
        ax4.set_xticklabels([r['image_filename'] for r in valid_results], rotation=45, ha='right')
        ax4.set_ylim(bottom=0)

        plt.tight_layout()
        plt.savefig(OUTPUT_PIPELINE_METRICS, dpi=300)
        plt.close()
        logging.info(f"Visualization saved to {OUTPUT_PIPELINE_METRICS}")

        total_prompt = sum(prompt_tokens)
        total_candidate = sum(candidate_tokens)
        total_api_time = sum(r.get('total_api_time_per_question', 0) for r in valid_results)
        logging.info("\nOverall Metrics for this Run:")
        logging.info(f"  Total Prompt Tokens Used: {total_prompt}")
        logging.info(f"  Total Candidate Tokens Generated: {total_candidate}")
        logging.info(f"  Total API Time Spent: {total_api_time:.2f} seconds")
        logging.info(f"  Success Rate: {success_rate:.1f}%")
        logging.info(f"  Accuracy Rate (of evaluated problems): {accuracy:.1f}%")

def preprocess_image(image_path):
    """Preprocess the image to improve LaTeX extraction."""
    img = Image.open(image_path).convert('L')
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2.0)
    img = img.filter(ImageFilter.SHARPEN)
    if max(img.size) > 800:
        img = img.resize((int(img.size[0] * 0.7), int(img.size[1] * 0.7)), Image.LANCZOS)
    return img

def extract_latex_from_image(image_path, max_retries=3):
    """Extracts LaTeX from an image using Gemini Vision."""
    logging.info(f"Attempting to extract LaTeX from {os.path.basename(image_path)} using Gemini Vision...")
    img = preprocess_image(image_path)
    prompt = """
    Analyze this image for mathematical equations and formulas. Your output MUST:
    - Identify and extract each distinct mathematical expression separately.
    - Label each equation with 'Equation X:' where X is a number (e.g., Equation 1:, Equation 2:).
    - Use $...$ for inline equations and $$...$$ for display equations.
    - If no equations are found, return the raw text content of the image.
    - Exclude markdown code blocks and include only the extracted content.
    - Ensure correct LaTeX for special characters, fractions, integrals, etc.
    - Place each labeled equation on a new line.
    Example:
    Equation 1: $ax^2 + bx + c = 0$
    Equation 2: $$\\int_0^1 x^3 \\sqrt{1 - x^2} \, dx = \\frac{2}{15}$$
    """
    all_extracted_latex = []
    for attempt in range(max_retries):
        try:
            start_time = time.time()
            response = call_gemini_with_rotation(prompt, img, model_name='gemini-2.0-flash', max_attempts=1)
            end_time = time.time()
            extracted_text = response.text.strip()
            if extracted_text:
                equations = []
                current_equation = []
                lines = extracted_text.split('\n')
                for line in lines:
                    line = line.strip()
                    if line.startswith("Equation"):
                        if current_equation:
                            equations.append("\n".join(current_equation).strip())
                        current_equation = [line]
                    elif line and current_equation:
                        current_equation.append(line)
                if current_equation:
                    equations.append("\n".join(current_equation).strip())
                if equations:
                    all_extracted_latex = equations
                    logging.info(f"Successfully extracted {len(equations)} LaTeX equations from {os.path.basename(image_path)} on attempt {attempt + 1}.")
                    return "\n".join(equations), end_time - start_time
                else:
                    logging.warning(f"Attempt {attempt + 1} failed: No valid equations segmented. Response: {extracted_text}")
                    return extracted_text, end_time - start_time
            else:
                logging.warning(f"Attempt {attempt + 1} failed: No LaTeX extracted. Response: {response.text}")
        except Exception as e:
            logging.error(f"Attempt {attempt + 1} failed with error: {e}")
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                logging.info(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
    logging.warning(f"All {max_retries} attempts failed for {os.path.basename(image_path)}.")
    debug_dir = os.path.join(TEMP_IMG_DIR, "debug")
    os.makedirs(debug_dir, exist_ok=True)
    debug_path = os.path.join(debug_dir, f"failed_{os.path.basename(image_path)}")
    Image.open(image_path).save(debug_path)
    logging.info(f"Saved failed image to {debug_path} for debugging.")
    return "No LaTeX found", 0

def extract_latex_from_pdf(pdf_path, page_selection="all"):
    """Extracts LaTeX from PDF pages, converting to images."""
    logging.info(f"Extracting LaTeX from PDF: {os.path.basename(pdf_path)} (pages: {page_selection})...")
    latex_expressions = []
    try:
        doc = fitz.open(pdf_path)
        pages_to_process = parse_page_selection(page_selection, doc.page_count)
        for i in pages_to_process:
            if i >= doc.page_count:
                logging.warning(f"Page {i+1} is out of bounds for PDF {os.path.basename(pdf_path)}. Skipping.")
                continue
            page = doc.load_page(i)
            pix = page.get_pixmap()
            img_filename = os.path.join(TEMP_IMG_DIR, f"{Path(pdf_path).stem}_page_{i+1}.png")
            pix.save(img_filename)
            logging.info(f"Saved temporary image: {img_filename}")
            extracted_latex, api_time = extract_latex_from_image(img_filename)
            if extracted_latex != "No LaTeX found":
                equations = extracted_latex.split('\n')
                for eq_idx, eq in enumerate(equations, 1):
                    if eq.strip().startswith("Equation"):
                        latex_expressions.append({
                            "page_number": i + 1,
                            "image_path": img_filename,
                            "latex": eq.strip(),
                            "equation_number": eq_idx,
                            "total_api_time_per_question": api_time
                        })
            else:
                latex_expressions.append({
                    "page_number": i + 1,
                    "image_path": img_filename,
                    "latex": extracted_latex,
                    "total_api_time_per_question": api_time
                })
        doc.close()
    except Exception as e:
        logging.error(f"Error processing PDF {pdf_path}: {e}")
    return latex_expressions

def compare_answers(answer1, answer2, rel_tol=1e-2):
    """Compare two answers with flexible comparison."""
    if answer1 == "No answer found" or answer2 == "No answer found":
        return False
    def unbox(ans):
        box_match = re.search(r"\\boxed\{([^}]+)\}", ans)
        return box_match.group(1) if box_match else ans
    a1 = unbox(answer1).strip()
    a2 = unbox(answer2).strip()
    if a1 == a2:
        return True
    def to_float(ans_str):
        try:
            if '/' in ans_str and ans_str.count('/') == 1:
                num_str, den_str = ans_str.split('/')
                num = float(num_str.strip())
                den = float(den_str.strip())
                if den == 0: return None
                return num / den
            ans_str_cleaned = ans_str.lower().replace(' ', '')
            if 'e' in ans_str_cleaned or 'x10^' in ans_str_cleaned or '*10^' in ans_str_cleaned:
                ans_str_for_eval = ans_str_cleaned.replace('x10^', 'e').replace('*10^', 'e')
                return float(ans_str_for_eval)
            if ans_str_cleaned == 'pi': return math.pi
            if ans_str_cleaned == 'e': return math.e
            return float(ans_str.replace(',', ''))
        except ValueError:
            return None
    val1 = to_float(a1)
    val2 = to_float(a2)
    if val1 is not None and val2 is not None:
        return math.isclose(val1, val2, rel_tol=rel_tol)
    return a1.lower() == a2.lower()

def call_gemini_with_rotation(prompt, image_path, model_name='gemini-2.0-flash', max_attempts=5):
    """Call Gemini API with key rotation and retry logic."""
    last_exception = None
    for attempt in range(max_attempts):
        try:
            api_key, idx = api_key_rotator.get_key()
            genai.configure(api_key=api_key)
            model = genai.GenerativeModel(model_name)
            if isinstance(image_path, Image.Image):
                response = model.generate_content([prompt, image_path])
            else:
                response = model.generate_content([prompt, Image.open(image_path)])
            return response
        except Exception as e:
            if "429" in str(e) or "quota" in str(e).lower():
                retry_after = None
                api_key_rotator.mark_bad_key(idx, retry_after)
                last_exception = e
                if attempt < max_attempts - 1:
                    wait_time = 2 ** attempt
                    logging.info(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                continue
            else:
                raise
    raise Exception(f"Failed after {max_attempts} attempts: {last_exception}")

def generate_problem_with_solutions(image_path, extracted_latex, max_attempts=5):
    """Generate STEM problem and solutions from LaTeX."""
    filename = os.path.basename(image_path)
    if extracted_latex == "No LaTeX found" or not extracted_latex.strip():
        return {
            "image_filename": filename,
            "error": "No LaTeX extracted",
            "status": "failed"
        }
    start_time = time.time()
    for attempt in range(max_attempts):
        try:
            if attempt < 2:
                prompt = f"""
                Given: {extracted_latex}
                Generate a STEM problem in STRICT JSON format with:
                - One PhD-level problem
                - Two solution approaches
                - Same final answer
                - Proper LaTeX escaping (DOUBLE backslashes)
                Output MUST be valid JSON with this EXACT structure:
                {{
                    "q": "Problem statement",
                    "s1": "Solution 1 steps",
                    "a1": "\\boxed{{answer}}",
                    "s2": "Solution 2 steps",
                    "a2": "\\boxed{{same_answer}}",
                    "cat": "Category",
                    "sub": "Subcategory"
                }}
                """
            else:
                prompt = f"""
                Given: {extracted_latex}
                Output this EXACT JSON:
                {{
                    "q": "Evaluate this expression",
                    "s1": "Solution steps",
                    "a1": "\\boxed{{result}}",
                    "s2": "Same as above",
                    "a2": "\\boxed{{result}}",
                    "cat": "Math",
                    "sub": "General"
                }}
                """
            response = call_gemini_with_rotation(prompt, image_path, model_name='gemini-2.0-flash', max_attempts=1)
            response_text = response.text
            problem_data = None
            for clean_attempt in range(3):
                try:
                    cleaned = clean_json_response(response_text)
                    problem_data = json.loads(cleaned)
                    if not all(k in problem_data for k in ['q', 's1', 'a1', 's2', 'a2']):
                        raise ValueError("Missing required keys")
                    if not compare_answers(problem_data['a1'], problem_data['a2']):
                        raise ValueError("Answer mismatch")
                    break
                except Exception as e:
                    if clean_attempt == 2:
                        raise
                    continue
            end_time = time.time()
            return {
                "image_filename": filename,
                "extracted_latex": extracted_latex,
                "generated_question_llm1": problem_data['q'],
                "llm1_solution": problem_data['s1'],
                "llm1_final_answer": problem_data['a1'],
                "llm2_proposed_solution": problem_data['s2'],
                "llm2_final_answer_extracted": problem_data['a2'],
                "judge_evaluation": "Correct",
                "api_answer_status": "yes",
                "category": problem_data.get('cat', 'Mathematics'),
                "subcategory": problem_data.get('sub', 'General'),
                "total_api_time_per_question": end_time - start_time,
                "total_prompt_tokens_per_question": response.prompt_token_count if hasattr(response, 'prompt_token_count') else 0,
                "total_candidate_tokens_per_question": response.candidate_token_count if hasattr(response, 'candidate_token_count') else 0
            }
        except Exception as e:
            logging.warning(f"Attempt {attempt+1} failed: {str(e)[:100]}...")
            if attempt < max_attempts - 1:
                time.sleep(2 ** attempt)
            continue
    return {
        "image_filename": filename,
        "error": f"Failed after {max_attempts} attempts",
        "status": "failed"
    }

def get_input_files():
    """Gathers input files from INPUT_DIR, handling images and PDFs."""
    input_files = []
    logging.info(f"Scanning for input files in {INPUT_DIR}...")
    for root, _, files in os.walk(INPUT_DIR):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                extracted_latex, api_time = extract_latex_from_image(file_path)
                input_files.append({
                    "image_path": file_path,
                    "latex": extracted_latex,
                    "total_api_time_per_question": api_time
                })
            elif file.lower().endswith('.pdf'):
                logging.info(f"Found PDF: {file_path}. Extracting pages as images...")
                extracted_pages = extract_latex_from_pdf(file_path, page_selection=PDF_PAGE_SELECTION)
                input_files.extend(extracted_pages)
            else:
                logging.warning(f"Skipping unsupported file type: {file_path}")
    if not input_files:
        logging.warning(f"No image or PDF files found in {INPUT_DIR}.")
    return sorted(input_files, key=lambda input_file: input_file['image_path'])

def save_results(results):
    """Saves the processed results to CSV and JSON files."""
    if not results:
        logging.warning("No results to save.")
        return
    successful_results = [r for r in results if isinstance(r, dict) and 'status' not in r]
    try:
        with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        logging.info(f"Results saved to {OUTPUT_JSON}")
    except Exception as e:
        logging.error(f"Error saving results to JSON: {e}")
    if successful_results:
        try:
            fieldnames = list(successful_results[0].keys())
            with open(OUTPUT_CSV, 'w', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                for row in successful_results:
                    writer.writerow(row)
            logging.info(f"Results saved to {OUTPUT_CSV}")
        except Exception as e:
            logging.error(f"Error saving results to CSV: {e}")
    else:
        logging.warning("No successful results to save to CSV.")

def main():
    """Main function to orchestrate the problem generation pipeline."""
    logging.info("Starting problem generation pipeline...")
    pipeline_start_time = time.time()

    # Check API key cooldowns
    now = time.time()
    max_cooldown = max(api_key_rotator.cooldown, default=0)
    if now < max_cooldown:
        wait_time = max_cooldown - now
        logging.info(f"API keys in cooldown for {wait_time:.2f} seconds. Waiting...")
        time.sleep(wait_time)

    input_files = get_input_files()
    if not input_files:
        logging.error("No input files found in the specified input directory.")
        return

    results = []
    files_to_process = input_files[:MAX_PROBLEMS]
    logging.info(f"Found {len(input_files)} input files. Processing up to {len(files_to_process)}.")
    max_workers = min(len(api_key_rotator.api_keys), 4)
    logging.info(f"Using {max_workers} workers for parallel processing.")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {
            executor.submit(generate_problem_with_solutions, file_info['image_path'], file_info['latex']): file_info
            for file_info in files_to_process
        }
        for i, future in enumerate(as_completed(future_to_file), 1):
            file_info = future_to_file[future]
            logging.info(f"[{i}/{len(files_to_process)}] Processing {os.path.basename(file_info['image_path'])}")
            try:
                result = future.result()
                results.append(result)
                if 'status' not in result:
                    logging.info(f"  Generated problem: {result['generated_question_llm1'][:70]}...")
                    logging.info(f"  LLM1 answer: {result['llm1_final_answer']}")
                    logging.info(f"  LLM2 answer: {result['llm2_final_answer_extracted']}")
                    logging.info(f"  Evaluation: {result['judge_evaluation']}")
                else:
                    logging.warning(f"  Failed to process {os.path.basename(file_info['image_path'])}: {result.get('error', 'Unknown error')}")
                # Add delay for single-key scenarios
                if len(api_key_rotator.api_keys) == 1:
                    time.sleep(12)  # 5 requests/min = 12s/request
            except Exception as e:
                logging.error(f"Exception for {os.path.basename(file_info['image_path'])}: {e}")
                results.append({
                    "image_filename": os.path.basename(file_info['image_path']),
                    "error": f"Processing failed: {str(e)}",
                    "status": "failed"
                })

    if results:
        save_results(results)
        success_count = len([r for r in results if isinstance(r, dict) and 'status' not in r])
        logging.info(f"Pipeline finished. Successfully processed {success_count}/{len(results)} problems.")
        visualizer = ProblemVisualizer(OUTPUT_DIR)
        visualizer.generate_metrics_chart(results)
    else:
        logging.warning("No valid results generated from any input file.")

    total_pipeline_time = time.time() - pipeline_start_time
    logging.info(f"Total pipeline execution time: {total_pipeline_time:.2f} seconds.")

if __name__ == "__main__":
    start_time = time.time()
    main()
    end_time = time.time()
    print("\n\n\n\n Total Time: ", end_time - start_time)

ERROR:root:No valid API keys found in GEMINI_API_KEYS. Please set it in Colab Secrets or environment.
ERROR:root:Attempt 1 failed with error: cannot access local variable 'idx' where it is not associated with a value
ERROR:root:Attempt 2 failed with error: cannot access local variable 'idx' where it is not associated with a value
ERROR:root:Attempt 3 failed with error: cannot access local variable 'idx' where it is not associated with a value
